# PTDF with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Sourabh Dalvi

## Introduction

PowerSimulations.jl supports linear PTDF optimal power flow formulation. This example shows a
single multi-period optimization of economic dispatch with a linearilzed DC-OPF representation of
using PTDF power flow and how to extract duals values or locational marginal prices for energy.

## Dependencies
We can use the same RTS data and some of the initialization as in
[OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "3_PowerSimulations_examples", "01_operations_problems.jl"));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages

Since we'll be retreving duals, we need a solver that returns duals values
here we use Ipopt.

In [2]:
using Ipopt
solver = optimizer_with_attributes(Ipopt.Optimizer)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[])

In the [OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define an economic dispatch (linear generation decisions) with
linear DC-OPF using PTDF network representation.
So, starting with the network, we can select from _almost_ any of the endpoints on this
tree:

In [3]:
TypeTree(PSI.PM.AbstractPowerModel,  init_expand = 10, scopesep="\n")

PowerModels
AbstractPowerModel
├──PowerModels
│  AbstractACRModel
│  ├──PowerModels
│  │  ACRPowerModel
│  └──PowerModels
│     AbstractIVRModel
│     └──PowerModels
│        IVRPowerModel (0 children)
├──PowerModels
│  AbstractACPModel
│  └──PowerModels
│     ACPPowerModel
├──PowerModels
│  AbstractConicModel
│  ├──PowerModels
│  │  AbstractWRConicModel
│  │  └──PowerModels
│  │     AbstractSOCWRConicModel (1 children)
│  └──PowerModels
│     AbstractWRMModel
│     └──PowerModels
│        AbstractSDPWRMModel (2 children)
├──PowerModels
│  AbstractWRModel
│  ├──PowerModels
│  │  AbstractQCWRModel
│  │  ├──PowerModels
│  │  │  AbstractQCRMPowerModel (1 children)
│  │  └──PowerModels
│  │     AbstractQCLSModel (1 children)
│  └──PowerModels
│     AbstractSOCWRModel
│     └──PowerModels
│        SOCWRPowerModel (0 children)
├──PowerModels
│  AbstractLPACModel
│  └──PowerModels
│     AbstractLPACCModel
│     └──PowerModels
│        LPACCPowerModel (0 children)
├──PowerModels
│  AbstractBFModel
│  ├──PowerModels
│  │  AbstractBFAModel
│  │  └──PowerModels
│  │     BFAPowerModel (0 children)
│  ├──PowerModels
│  │  AbstractBFConicModel
│  │  └──PowerModels
│  │     AbstractSOCBFConicModel (1 children)
│  └──PowerModels
│     AbstractBFQPModel
│     └──PowerModels
│        AbstractSOCBFModel (1 children)
├──PowerModels
│  AbstractActivePowerModel
│  ├──PowerSimulations
│  │  AreaBalancePowerModel
│  ├──PowerSimulations
│  │  CopperPlatePowerModel
│  └──PowerModels
│     AbstractDCPModel
│     ├──PowerModels
│     │  AbstractDCPLLModel (1 children)
│     ├──PowerModels
│     │  AbstractNFAModel (1 children)
│     ├──PowerModels
│     │  DCPPowerModel (0 children)
│     ├──PowerModels
│     │  AbstractDCMPPModel (1 children)
│     └──PowerSimulations
│        StandardPTDFModel (0 children)
└──PowerModels
   AbstractACTModel
   └──PowerModels
      ACTPowerModel

For now, let's just choose a standard PTDF formulation.

In [4]:
ed_template = template_economic_dispatch(network = StandardPTDFModel)


Operations Problem Specification

  transmission:  PowerSimulations.StandardPTDFModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = PowerSimulations.ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirBudget
      Loads:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        

Currently  energy budget data isn't stored in the RTS-GMLC dataset.

In [5]:
ed_template.devices[:Hydro] = DeviceModel(HydroEnergyReservoir, HydroDispatchRunOfRiver)

PowerSimulations.DeviceModel{HydroEnergyReservoir,PowerSimulations.HydroDispatchRunOfRiver}(HydroEnergyReservoir, PowerSimulations.HydroDispatchRunOfRiver, nothing, PowerSimulations.ServiceModel[])

Calculate the PTDF matrix.

In [6]:
PTDF_matrix = PTDF(sys)

PowerNetworkMatrix
:
  0.0844408   -0.0535817   -0.0534019   …  0.0   0.112352      0.00242657
  0.10766     -0.0105107   -0.0104716      0.0   0.123927      0.00400695
  0.0473004    0.0392692    0.0393108      0.0   0.0635142    -0.119254
 -0.00287493  -0.00978349  -0.0108591      0.0  -0.00211729    0.000288697
  0.195809    -0.0369726   -0.0368496      0.0   0.0498628     0.000715747
  0.0497557    0.0330662    0.0330439   …  0.0   0.0634895     0.0898541
  0.00931269   0.0378806    0.0363124      0.0   0.0068585    -0.000935171
  0.220143     0.0340384    0.0339698      0.0   0.406336      0.0187527
 -0.0215194   -0.0840679   -0.0843584      0.0  -0.00807837    0.02744
  0.0461277    0.0179094    0.0178338      0.0   0.0623766     0.0963081
  ⋮                                     ⋱  ⋮                  
  0.198051    -0.00997791  -0.00994383     0.0   0.0367839     0.00100157
  0.0117709   -0.0125304   -0.0125877      0.0   0.0272047    -0.0129641
  0.00233406   0.0089126    0.0089

Now we can build a 4-hour economic dispatch / PTDF problem with the RTS data.
Here, we have to pass the keyword argument `constraint_duals` to OperationsProblem
with the name of the constraint for which duals are required for them to be returned in the results.

In [7]:
problem = OperationsProblem(
    EconomicDispatchProblem,
    ed_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
    constraint_duals = [:CopperPlateBalance, :network_flow],
    PTDF = PTDF_matrix,
)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/base.jl:282
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/devices/thermal_generation.jl:595
┌ Warning: The data doesn't include devices of type Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/0nHyl/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  PowerSimulations.StandardPTDFModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = ThermalStandard
        formulation = PowerSimulations.ThermalRampLimited
      DistRE:
        device_type = RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        devic

And solve the problem and collect the results

In [8]:
res = solve!(problem)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 61235.2 - 0.02 seconds
Cgl0003I 142 fixed, 0 tightened bounds, 210 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 149 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 81 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 45 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 17 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0004I processed model has 1412 rows, 3500 columns (612 integer (612 of which binary)) and 7784 elements
Cbc0045I Trying just fixing integer variables (and fixingish SOS).
Cbc0045I MIPStart provided solution wi

Results
========

Variables
=========

Fp__HVDCLine: (4, 1)
γ⁻__P: (4, 73)
P__ThermalStandard: (4, 76)
P__RenewableDispatch: (4, 30)
Fp__TapTransformer: (4, 15)
P__HydroEnergyReservoir: (4, 19)
γ⁺__P: (4, 73)
Fp__Line: (4, 105)


Optimizer Log
-------------
        timed_solve_time = 1.061238115
        solve_bytes_alloc = 25462606
        solve_time = 0.8524510860443115
        obj_value = 1.94007667171723e7
        solver = Ipopt
        sec_in_gc = 0.0
        dual_status = FEASIBLE_POINT
        primal_status = FEASIBLE_POINT
        termination_status = LOCALLY_SOLVED


Total Cost: OBJECTIVE_FUNCTION = 1.94007667171723e7

Here we collect the dual values from the results for the `CopperPlateBalance` and `network_flow`
constraints. In the case of PTDF network formulation we need to compute the final LMP for each bus in the system by
subtracting the duals (μ) of `network_flow` constraint multipled by the PTDF matrix
from the  dual (λ) of `CopperPlateBalance` constraint.
Note:we convert the results from DataFrame to Array for ease of use.

In [9]:
λ = convert(Array, res.dual_values[:CopperPlateBalance])
μ = convert(Array, res.dual_values[:network_flow])

4×120 Array{Float64,2}:
 -2.1222e-6   -1.58489e-6  -1.72109e-6  …  -1.27447e-6  -2.06555e-6
 -2.17033e-6  -1.60185e-6  -1.7781e-6      -1.21956e-6  -2.12111e-6
 -2.20758e-6  -1.61529e-6  -1.82913e-6     -1.18372e-6  -2.15635e-6
 -2.23938e-6  -1.62141e-6  -1.90767e-6     -1.14197e-6  -2.18695e-6

Here we create Dict to store the calculate congestion component of the LMP which is a product of μ and the PTDF matrix.

In [10]:
buses = get_components(Bus, sys)
congestion_lmp = Dict()
for bus in buses
    congestion_lmp[get_name(bus)] = μ * PTDF_matrix[:, get_number(bus)]
end
congestion_lmp = DataFrame(congestion_lmp)

,Abel,Adams,Adler,Agricola,Aiken,Alber,Alder
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-3.41569e-8,-9.06212e-7,-2.01344e-7,-3.9962e-6,-3.85315e-6,-6.95364e-6,9.60362e-6
2,-3.87865e-7,-1.31838e-6,-2.69864e-7,-4.35011e-6,-4.18129e-6,-7.33061e-6,9.56838e-6
3,-6.53542e-7,-1.62258e-6,-3.40933e-7,-4.60204e-6,-4.41636e-6,-7.63447e-6,9.55733e-6
4,-1.07537e-6,-2.08096e-6,-5.30728e-7,-4.97512e-6,-4.77263e-6,-8.10939e-6,9.65538e-6


Finally here we get the LMP for each node in a lossless DC-OPF using the PTDF formulation.

In [11]:
LMP = λ .- congestion_lmp

,Abel,Adams,Adler,Agricola,Aiken,Alber,Alder,Alger,Ali
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
2,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
3,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6
4,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6,-1.0e6


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*